In [1]:
import sqlite3
import pandas as pd
import numpy as np
import json
from operator import itemgetter

In [2]:
connection = sqlite3.connect("test.db")
cursor = connection.cursor()

In [3]:
def read_query(query, cursor):
    cursor.execute(query)
    result = cursor.fetchall()
    attr_arr = cursor.description
    attr = []
    for val in attr_arr:
        attr.append(val[0])
    json_res = []
    for user in result:
        obj = {}
        for i in range(len(attr)):
            obj[attr[i]] = user[i]
        json_res.append(obj)
    
    return json_res

In [4]:
query = '''select students.regi, students.name, courses.code, courses.name as course_title, marks.marks
from students inner join marks on students.regi=marks.regi
inner join courses on marks.course_code=courses.code;'''  # same query as sql_e
result = read_query(query, cursor)
result[0:3]

[{'regi': 2121000001,
  'name': 'Adam',
  'code': 'CSE331',
  'course_title': 'Software Engineering and Design Patterns',
  'marks': 90.0},
 {'regi': 2121000001,
  'name': 'Adam',
  'code': 'CSE333',
  'course_title': 'Database Management Systems',
  'marks': 97.0},
 {'regi': 2121000001,
  'name': 'Adam',
  'code': 'CSE335',
  'course_title': 'Operating Systems',
  'marks': 93.5}]

## Write appropriate queries and manipulate the data using python to create a **tabulation** dictionary like below and **pysql_1.ipynb**

In [5]:
tabulation = {
    2121000001: {
      'Registration': 2121000001,
      'name': 'Adam',
      'CSE331': 90.0,
      'CSE333': 97.0,
      'CSE335': 93.5,
      'CSE365': 96.5,
      'CSE325': 88.5,
      'CSE345': 100.0
    },
    2121000002: {
      'Registration': 2121000002,
      'name': 'Alex',
      'CSE331': 62.0,
      'CSE333': 37.0,
      'CSE335': 49.5,
      'CSE365': 44.5,
      'CSE325': 53.5,
      'CSE345': 46.5
    },
    2121000003: {
      'Registration': 2121000003,
      'name': 'Aaron',
      'CSE331': 91.0,
      'CSE333': 75.0,
      'CSE335': 83.0,
      'CSE365': 83.0,
      'CSE325': 76.0,
      'CSE345': 91.0
    },
}

## Same as 2(a)

In [6]:
tabulation = {}
for marks in result:
    if marks['regi'] not in tabulation:
        tabulation[marks['regi']] = {
            'Registration': marks['regi'],
            'name': marks['name'],
        }
    tabulation[marks['regi']][marks['code']] = marks['marks']
# tabulation

## It is preferred if you do not touch the following code snippets

In [7]:
gpa_matrix = {}
credit = 4.00
for i in range(80, 39, -5):
    gpa_matrix[i] = credit
    credit -= 0.25
gpa_matrix[35] = 0
gpa_matrix


def getGPA(marks):
    scle = min(marks, 80)
    scle = max(scle, 35)
    scle -= scle%5

    return gpa_matrix[scle]

getGPA(95)

4.0

In [8]:
# Assumes all the courses have similar credits
def processStudentInfo(info):
    ret = {
        "cgpa": 0.0,
        "total": 0
    }
    for key in info:
        if key == "Registration" or key == "name":
            ret[key] = info[key]
            continue
        ret['cgpa'] += getGPA(info[key])
        ret['total'] += info[key]

    ret['cgpa'] = round(ret['cgpa']/(len(info.keys()) - 2), 2)
    return ret

processStudentInfo({
    'Registration': 2121000001,
    'name': 'Adam',
    'CSE331': 90.0,
    'CSE333': 97.0,
    'CSE335': 93.5,
    'CSE365': 96.5,
    'CSE325': 88.5,
    'CSE345': 100.0
})


{'cgpa': 4.0, 'total': 565.5, 'Registration': 2121000001, 'name': 'Adam'}

## Populate an array named **processed** which contains information in the format given below

In [10]:
# processed = [
#     {'cgpa': 4.0, 'total': 565.5, 'Registration': 2121000001, 'name': 'Adam'},
#     {'cgpa': 2.0, 'total': 293.0, 'Registration': 2121000002, 'name': 'Alex'},
#     {'cgpa': 3.92, 'total': 499.0, 'Registration': 2121000003, 'name': 'Aaron'}
# ]

processed = []

for student in tabulation:
    processed.append(processStudentInfo(tabulation[student]))
processed[0:3]

[{'cgpa': 4.0, 'total': 565.5, 'Registration': 2121000001, 'name': 'Adam'},
 {'cgpa': 2.0, 'total': 293.0, 'Registration': 2121000002, 'name': 'Alex'},
 {'cgpa': 3.92, 'total': 499.0, 'Registration': 2121000003, 'name': 'Aaron'}]

## Use the following 2 cells to show result

In [11]:
processed.sort(key=itemgetter('total'), reverse=True) # sort based on total marks first
processed.sort(key=itemgetter('cgpa'), reverse=True)  # finally sort based on cgpa

In [12]:
rank = pd.DataFrame(processed)
rank[["Registration", "name", "cgpa", 'total']].style.hide_index()

Registration,name,cgpa,total
2121000001,Adam,4.000000,565.500000
2121000083,Tandy,4.000000,557.000000
2121000024,Paul,4.000000,535.000000
2121000068,Ramachandran,4.000000,505.000000
2121000116,Celedon,3.960000,506.000000
2121000032,Victor,3.960000,495.000000
2121000003,Aaron,3.920000,499.000000
2121000078,Sonderling,3.920000,486.000000
2121000120,Darnell,3.880000,488.000000
2121000048,Haworth,3.830000,481.000000
